### This step-by-step tutorial showcases our recently formulated distributed Operator Inference (dOpInf) algorithm in a two-dimensional transient flow past a circular cylinder scenario governed by the 2D incompressible Navier-Stokes equations 
### The goal of this tutorial is to provide a detailed description of dOpInf and to guide users on practical details and facilitate a seamless integration with complex applications

### Problem description
* #### we consider the canonical problem of two-dimensional transient flow past a circular cylinder, a widely-used benchmark in computational fluid dynamics and reduced-order modeling
* #### the fluid flow is governed by the 2D incompressible Navier-Stokes equations
#### $$ 
\begin{align*}
\partial_t \mathbf{u} + \nabla \cdot (\mathbf{u} \otimes \mathbf{u}) & = \nabla p + Re^{-1}\Delta \mathbf{u} \\
\nabla \cdot \mathbf{u} & = 0,
\end{align*}
$$ 
#### where $p \in \mathbb{R}$ denotes the pressure, $\mathbf{u} = (u_x, u_y)^\top \in \mathbb{R}^2$ denotes the $x$ and $y$ components of the velocity vector, and $Re$ denotes the Reynolds number
* #### the problem setup, geometry, and parameterization are based on the [DFG 2D-3 benchmark in the FeatFlow suite](https://wwwold.mathematik.tu-dortmund.de/~featflow/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html)
* #### this setup uses $Re = 100$, which represents a value that is above the critical Reynolds number for the onset of the two-dimensional vortex shedding
* #### the physical domain is depicted in the figure below:
<img src="images/navier_stokes_cylinder_geometry.png" width="700" class="center">

### Numerical discretization
* #### we solve the 2D Navier Stokes equations numerically in double precision using finite elements with the [FEniCS](https://fenicsproject.org/) software package
* #### our implementation provided in the [**generate_high_fidelity_data.py**](./high_fidelity_code/generate_high_fidelity_data.py) script closely follows this [tutorial](https://fenicsproject.org/pub/tutorial/html/._ftut1009.html)
* #### we use piecewise quadratic finite elements for the velocity and piecewise linear elements for the pressure
* #### the resulting system of equations is large and sparse, making direct solvers impractical
* #### we employ iterative Krylov subspace methods with preconditioning for improved performance
* #### we utilize the biconjugate gradient stabilized (BiCGstab) method and the conjugate gradient (CG) method

### Acquiring the training data
* #### the finite element mesh contains $n_x = 146,339$ degrees of freedom
* #### the discretized governing equations are integrated over the time interval $[0, 10]$ seconds in increments of $\Delta t = 2.5 \times 10^{-5}$ seconds
* #### this amounts to a total of $40,000$ time instants of the pressure and velocity fields
* #### in the model reduction experiments that follow, we simplify the model by omitting the pressure term, which is justified by the fact that, for both transient and periodic flow regimes, the pressure can be implicitly determined from the velocity field through the incompressibility constraint
* #### this means that the number of state variables is $n_s = 2$
* #### therefore, the size of one data snapshot is $n = n_s \times n_x = 292,678$
* #### the target time horizon is $[4, 10]$ seconds, corresponding to the periodic regime
* #### training data are collected over $[4, 7]$ seconds, whereas the remainder of the target time horizon is used for predictions beyond training
* #### storing the full velocity fields over the training horizon would require approximately $28$ GB of storage, exceeding the capabilities of a typical personal computer
* #### we downsample the training data by a factor of $20$, reducing its storage footprint to a manageable size of around $1.4$ GB for $n_t = 600$ downsampled snapshots
* #### the training data were saved to disk in a single HDF5 file [**velocity_training_snapshots.h5**](./navier_stokes_benchmark/velocity_training_snapshots.h5)
* #### this file contains two datasets, $u\_x$ and $u\_y$, for the two downsampled velocity components of dimension $146,339 \times 600$
* #### this amounts to a snapshot matrix $\mathbf{S} \in \mathbb{R}^{n \times n_t} = \mathbb{R}^{292,678 \times 600}$

### Steps to implement the dOpInf algorithm
* #### let $p \in \mathbb{N}_{\geq 2}$ denote the number of compute cores to perform dOpInf
* #### this tutorial focuses on a dOpInf implementation in Python using the Message Passing Interface (MPI) library [mpi4py](https://mpi4py.readthedocs.io/en/stable/)
* #### for convenience, we will use in the following the notation $i = 0, 1, \dots, p-1$ to denote both the $i$th core and its corresponding MPI rank
  
#### Step I: Parallel training data loading
* #### we start by creating a parallel client

In [ ]:
import ipyparallel as ipp
rc   = ipp.Client()
view = rc[:]

* #### we then import the necessary libraries for our computations, define key variables, and initialize the standard MPI communicator
* #### the rank of each compute core is given by the variable <em>rank</em> ranging from $0$ to $p-1$, whereas the total size of the communicator ($p$ in our case) is given by the variable <em>size</em>
* #### note that each cell that executes parallel code must start with the parallel magic command <em>%%px</em>

In [ ]:
%%px
import numpy as np
import h5py as h5
from mpi4py import MPI

In [ ]:
%%px
# DoF setup
ns 	= 2
n 	= 292678 
nx  = int(n/ns)

# number of training snapshots
nt = 600

# state variable names
state_variables = ['u_x', 'u_y']

# path to the HDF5 file containing the training snapshots
H5_training_snapshots = 'navier_stokes_benchmark/velocity_training_snapshots.h5'

# number of time instants over the time domain of interest (training + prediction)
nt_p = 1200

# MPI initialization
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

* #### we then load the training data in parallel
* #### the user must choose a strategy for distributing the full training dataset $\mathbf{S} \in \mathbb{{R}}^{n \times n_t}$ into $p$ non-overlapping blocks $\mathbf{S}_i \in \mathbb{R}^{n_i \times n_t}$ for $i = 0, 1, \dots, p-1$ such that $\sum_{i = 0}^{p-1} n_i = n$
* #### the splitting strategy depends on the target parallel architecture and whether training data manipulations are necessary
* #### in the code below, we distribute the number of spatial degrees of freedom $n_x$ into $p$ independent components $n_{x, i}$ such that $\sum_{i=0}^{p-1} n_{x, i} = 146,339$
* #### moreover, $n_i = 2 n_{x, i}$ and $\sum_{i=0}^{p-1} n_{i} = 292,678$
* #### this corresponds to decomposing the spatial domain into $p$ non-overlapping subdomains
* #### this splitting scheme allows to efficiently perform data transformations in parallel
* #### each rank then reads the two velocity components corresponding to its subdomain into a snapshot matrix denoted by <em>Q\_rank</em> of size $2 n_{x, i} \times n_t = n_{i} \times n_t$; we explain below why we denote the snapshot matrix on each rank by <em>Q\_rank</em> instead of <em>S\_rank</em>

In [ ]:
%%px
def distribute_nx(rank, nx, size):
	"""
 	distribute_nx distributes the spatial DoF nx into chunks of size nx_i such that 
 	\sum_{i=0}^{p-1} nx_i = nx where p is the number of used compute cores

 	:rank: 	the MPI rank i = 0, 1, ... , p-1 that will execute this function
 	:n_x: 	number of DoF used for spatial discretization
 	:size: 	size of the MPI communicator (p in our case)
 	
 	:return: the start and end index of the local DoF, and the number of local DoF for each rank
 	"""

	nx_i_equal = int(nx/size)

	nx_i_start = rank * nx_i_equal
	nx_i_end   = (rank + 1) * nx_i_equal

	if rank == size - 1 and nx_i_end != nx:
		nx_i_end += nx - size*nx_i_equal

	nx_i = nx_i_end - nx_i_start

	return nx_i_start, nx_i_end, nx_i

# the start and end indices, and the total number of snapshots for each MPI rank
nx_i_start, nx_i_end, nx_i = distribute_nx(rank, nx, size)
        
# allocate memory for the snapshot data corresponding to each MPI rank
# the full snapshot data has been saved to disk in HDF5 format
Q_rank = np.zeros((ns * nx_i, nt))
with h5.File(H5_training_snapshots, 'r') as file:

    for j in range(ns):
        Q_rank[j*nx_i : (j + 1)*nx_i, :] = \
		          file[state_variables[j]][nx_i_start : nx_i_end, :]

file.close()

### Step II: Parallel training data manipulations
* #### in the next step, we perform any necessary data manipulations, which can vary depending on the specific problem
* #### this might include variable transformations for scenarios with non-polynomial governing equations or scaling transformations for those involving multiple states with varying scales
* #### if data transformations are used, we denote by $\mathbf{Q}_i \in \mathbb{R}^{m_i \times n_t}$ the \emph{transformed} snapshot data on each compute core, with $\sum_{i=0}^{p-1} m_i = m \geq n$ denoting the transformed snapshot dimension
* #### note that $m$ exceeds $n$ when the number of lifted variables exceeds the number of original state variables
* #### if data transformations are not required, we employ $\mathbf{S}_i$ as is and use the notation $\mathbf{Q}_i = \mathbf{S}_i$ and $m_i = n_i$ for convenience
* #### for the considered 2D Navier-Stokes example, the only employed data transformation is snapshot centering with respect to the temporal mean over the training time horizon
* #### since the data splitting was done over the spatial domain, performing centering with respect to the mean is trivial in an MPI program, as illustrated in the code snippet below

In [ ]:
%%px
# compute the temporal mean of each variable on each rank
temporal_mean_rank  = np.mean(Q_rank, axis=1)
# center (in place) each variable with respect to its temporal mean on each rank
Q_rank              -= temporal_mean_rank[:, np.newaxis]

### Step III: Parallel dimensionality reduction
* #### in the next step, we perform parallel dimensionality reduction, which represent the high-dimensional (transformed) snapshot data with global dimension $m$ in a lower-dimensional space of dimension $r$ such that $r \ll m$
* #### here, the reduced space is a linear subspace, spanned by the column vectors forming the $r$-dimensional POD basis
* #### this step is typically the most computationally and memory-intensive part of the standard, serial OpInf formulation
* #### starting from the method of snapshots, we can scalably and efficiently represent the high-dimensional snapshot data in the low-dimensional subspace spanned by the $r$-dimensional POD basis vectors without explicitly having to compute the POD basis and without introducing approximations by using, for example, a randomized SVD technique
* #### we start by computing the Gram matrices $\mathbf{D}_i = \mathbf{Q}_i^\top \mathbf{Q}_i \in \mathbb{R}^{n_t \times n_t}$ on each core, followed by their summation $\mathbf{D} = \sum_{i=1}^p \mathbf{D}_i$ obtained via a colective parallel reduction
* #### consider the thin singular value decomposition (SVD) of $\mathbf{Q}$ computed at a cost in $\mathcal{O}(mn_t^2)$:
$$ \begin{equation}
    \mathbf{Q} = \mathbf{V} \mathbf{\Sigma} \mathbf{W}^\top,
\end{equation}
$$
#### where $\mathbf{V} \in \mathbb{R}^{m \times n_t}$ contains the left singular vectors, $\mathbf{\Sigma} \in \mathbb{R}^{n_t \times n_t}$ is a diagonal matrix comprising the singular values in non-decreasing order $\sigma_1 \geq \sigma_2 \geq \ldots \geq \sigma_{n_t}$, where $\sigma_j$ denotes the $j$th singular value, and $\mathbf{W} \in \mathbb{R}^{n_t \times n_t}$ contains the right singular vectors
* #### the rank-$r$ POD basis $\mathbf{V}_r \in \mathbb{R}^{m \times r}$ is obtained from the first $r$ columns of $\mathbf{V}$ corresponding to the $r$ largest singular values
* #### in the standard OpInf formulation, the low-dimensional representation of $\mathbf{Q}$ in the linear subspace spanned by the columns of $\mathbf{V}_r$ is computed at a cost in $\mathcal{O}(r  m  n_t)$ as
$$
\begin{equation}
\hat{\mathbf{Q}} = \mathbf{V}_r^\top \mathbf{Q} \in \mathbb{R}^{r \times n_t}
\end{equation}
$$
* #### since the original snapshot data was partitioned non-overlappingly, it follows that
$$
\begin{equation*} 
    \mathbf{D} = \sum_{i=1}^p \mathbf{D}_i = \sum_{i=1}^p \mathbf{Q}_i^\top \mathbf{Q}_i = \mathbf{Q}^\top \mathbf{Q}.
\end{equation*}
$$
* #### we moreover have that
$$
\begin{equation} 
    \mathbf{D} = \mathbf{Q}^\top \mathbf{Q} = \mathbf{W} \mathbf{\Sigma} \mathbf{V}^\top \mathbf{V} \mathbf{\Sigma} \mathbf{W}^\top = \mathbf{W} \mathbf{\Sigma}^2 \mathbf{W}^\top \Rightarrow \mathbf{D} \mathbf{W} = \mathbf{W} \mathbf{\Sigma}^2,
\end{equation}
$$
#### which implies that the eigenvalues of $\mathbf{D}$ are the squared singular values of $\mathbf{Q}$, and the eigenvectors of $\mathbf{D}$ are equivalent (up to a sign change) to the right singular vectors of $\mathbf{Q}$
* #### each core then computes the eigenpairs $\{(\lambda_k, \mathbf{u}_k)\}_{k=1}^{n_t}$ of $\mathbf{D}$, where $\lambda_k$ are the real and non-negative eigenvalues and $\mathbf{u}_k \in \mathbb{R}^{n_t}$ denote the corresponding eigenvectors
* #### we also ensure that the eigenpairs are arranged such that $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_{n_t}$
* #### the rank-$r$ POD basis can be also computed as
$$
\begin{equation}
    \mathbf{V}_r = \mathbf{Q} \mathbf{W}_r \mathbf{\Sigma}_r^{-1} = \mathbf{Q} \mathbf{U}_r \mathbf{\Lambda}_r^{-\frac{1}{2}}, 
\end{equation}
$$
#### where $\mathbf{U}_r = \begin{bmatrix} \mathbf{u}_1 \vert \mathbf{u}_2 \vert \dots \vert \mathbf{u}_r \end{bmatrix}$ and $\mathbf{\Lambda}_r = \mathrm{diag}(\lambda_1, \lambda_2, \ldots, \lambda_r)$
* #### however, the above expression implies that
$$
\begin{equation}
    \hat{\mathbf{Q}} = \mathbf{V}_r^\top \mathbf{Q} = \left(\mathbf{Q} \mathbf{U}_r \mathbf{\Lambda}_r^{-\frac{1}{2}}\right)^\top \mathbf{Q} = \mathbf{T}_r^\top \mathbf{Q}^\top \mathbf{Q} = \mathbf{T}_r^\top \mathbf{D},
\end{equation}
$$
#### where we used the notation $\mathbf{T}_r =  \mathbf{U}_r \mathbf{\Lambda}_r^{-\frac{1}{2}} \in \mathbb{R}^{n_t \times r}$
* #### therefore, the representation of the high-dimensional (transformed) snapshots in the low-dimensional linear subspace spanned by the rank-$r$ POD basis vectors can be efficiently computed in terms of two small matrices, $\mathbf{T}_r$ and $\mathbf{D}$, without explicitly requiring the POD basis
* #### in our implementation, we choose the reduced dimension, $r$, such that the total retained energy corresponding to the first $r$ POD modes is $99.96 \%$, that is,
$$
\begin{equation}
    \frac{\sum_{k=1}^{r} \sigma_k^2}{\sum_{k=1}^{n_t} \sigma_k^2} \geq 0.9996
\end{equation}
$$

In [ ]:
%%px
# compute the local Gram matrices on each rank
D_rank = np.matmul(Q_rank.T, Q_rank)

# compute the global Gram matrix via a parallel reduction of the local Gram matrices, and broadcast the result to all ranks
D_global = np.zeros_like(D_rank)
comm.Allreduce(D_rank, D_global, op=MPI.SUM)

# compute the eigendecomposition of the positive, semi-definite global Gram matrix
eigs, eigv = np.linalg.eigh(D_global)

# order eigenpairs by increasing eigenvalue magnitude
sorted_indices  = np.argsort(eigs)[::-1]
eigs            = eigs[sorted_indices]
eigv            = eigv[:, sorted_indices]

# define target retained energy for the dOpInf ROM
target_ret_energy = 0.9996

# compute retained energy for r bteween 1 and nt
ret_energy  = np.cumsum(eigs)/np.sum(eigs)
# determine the minimum value of r that exceeds the prescribed retained energy threshold
r           = np.argmax(ret_energy > target_ret_energy) + 1

# compute the auxiliary Tr matrix
Tr_global 	= np.matmul(eigv[:, :r], np.diag(eigs[:r]**(-0.5)))
# compute the low-dimensional representation of the high-dimensional transformed snapshot data
Qhat_global = np.matmul(Tr_global.T, D_global)

### Step IV: Parallel reduced operator learning via Operator Inference
* #### the Navier-Stokes equations have only linear and quadratic terms in the governing equations
* #### we additionally have a constant term introduced into the ROM, due to centering
* #### since the training dataset was severly downsampled in time, we employ the time-discrete formulation of OpInf
* #### the goal is to determine the reduced operators $\hat{\mathbf{c}} \in \mathbb{R}^{r}, \hat{\mathbf{A}} \in \mathbb{R}^{r\times r}$, and $\hat{\mathbf{H}} \in \mathbb{R}^{r\times r^2}$ defining the discrete quadratic ROM
$$
\begin{equation} 
    \hat{\mathbf{q}}[k + 1] = \hat{\mathbf{A}}\hat{\mathbf{q}}[k] + \hat{\mathbf{H}}\left(\hat{\mathbf{q}}[k] \otimes \hat{\mathbf{q}}[k] \right) + \hat{\mathbf{c}}
\end{equation}
$$
#### that best match the projected snapshot data in a minimum residual sense by solving the linear least-squares minimization
$$
\begin{equation}
    \mathop{\mathrm{argmin}}_{\hat{\mathbf{O}}} \left\lVert \hat{\mathbf{D}}\hat{\mathbf{O}}^{\top} - \hat{\mathbf{Q}}_2^\top \right\rVert_F^2 + \beta_{1} \left(\left\lVert\hat{\mathbf{A}}\right\rVert_F^2 + \left\lVert\hat{\mathbf{c}}\right\rVert_F^2\right) + \beta_{2} \left\lVert\hat{\mathbf{H}}\right\rVert_F^2,
\end{equation}
$$
#### where  $\hat{\mathbf{O}} =
\begin{bmatrix}
\hat{\mathbf{A}} \, \vert \, \hat{\mathbf{H}} \, \vert \, \hat{\mathbf{c}}
\end{bmatrix}
\in \mathbb{R}^{r \times (r + r^2 + 1)}$ denotes the unknown operators and  $\hat{\mathbf{D}} =
\begin{bmatrix}
\hat{\mathbf{Q}}_1^\top \, \vert \, \hat{\mathbf{Q}}_1^\top \otimes \hat{\mathbf{Q}}_1^\top \, \vert \, \hat{\mathbf{1}}_{n_t - 1}
\end{bmatrix}
\in \mathbb{R}^{(n_t - 1) \times (r + r^2 +1)}$ the OpInf data, $F$ denotes the Frobenius norm, and
\begin{equation} 
    \hat{\mathbf{Q}}_{1} =
     \begin{bmatrix}
\vert & \vert & & \vert\\
     \hat{\mathbf{q}}_1 &
     \hat{\mathbf{q}}_2 &
     \ldots &
     \hat{\mathbf{q}}_{n_t - 1}\\
     \vert & \vert & & \vert
     \end{bmatrix} \in \mathbb{R}^{r \times n_t-1} \quad \text{and} \quad 
     \hat{\mathbf{Q}}_{2} =
     \begin{bmatrix}
\vert & \vert & & \vert\\
     \hat{\mathbf{q}}_2 &
     \hat{\mathbf{q}}_3 &
     \ldots &
     \hat{\mathbf{q}}_{n_t}\\
     \vert & \vert & & \vert
     \end{bmatrix} \in \mathbb{R}^{r \times n_t-1}
\end{equation}
* #### to address overfitting and other sources of error, we introduce regularization hyperparameters $\beta_{1}, \beta_{2} \in \mathbb{R}$
* #### we start with providing the code for four auxiliary functions that we will need later on

In [ ]:
%%px
def distribute_reg_pairs(rank, n_reg, size):
	"""
 	get_reg_params_per_rank returns the index of the first and last regularization pair for each MPI rank
 
 	:rank:    the MPI rank i = 0, 1, ... , p-1 that will execute this function
 	:n_reg:   total number of regularization parameter pairs
 	:size: 	  size of the MPI communicator (p in our case)

 	:return: the start and end indices of the regularization pairs for each MPI rank
 	"""

	nreg_i_equal = int(n_reg/size)

	start = rank * nreg_i_equal
	end   = (rank + 1) * nreg_i_equal

	if rank == size - 1 and end != n_reg:
		end += n_reg - size*nreg_i_equal

	return start, end

def compute_Qhat_sq(Qhat):
	"""
	compute_Qhat_sq returns the non-redundant terms in Qhat \otimes Qhat

	:Qhat: reduced snapshot data

	:return: the non-redundant in Qhat \otimes Qhat
	"""

	if len(np.shape(Qhat)) == 1:

	    r 		= np.size(Qhat)
	    prods 	= []
	    for i in range(r):
	        temp = Qhat[i]*Qhat[i:]
	        prods.append(temp)

	    Qhat_sq = np.concatenate(tuple(prods))

	elif len(np.shape(Qhat)) == 2:
	    K, r 	= np.shape(Qhat)    
	    prods 	= []
	    
	    for i in range(r):
	        temp = np.transpose(np.broadcast_to(Qhat[:, i], (r - i, K)))*Qhat[:, i:]
	        prods.append(temp)
	    
	    Qhat_sq = np.concatenate(tuple(prods), axis=1)

	else:
	    print('invalid input!')

	return Qhat_sq

def compute_train_err(Qhat_train, Qtilde_train):
	"""
	compute_train_err computes the OpInf training error

	:Qhat_train:   reference data computed by projecting the high-dimensional transformed snapshots
	:Qtilde_train: approximate data computed by the dOpInf ROM

	:return: the value of the training error
	"""
	train_err = \
    np.max(np.sqrt(np.sum( (Qtilde_train - Qhat_train)**2, axis=1) / np.sum(Qhat_train**2, axis=1)))

	return train_err

def solve_discrete_dOpInf_model(qhat0, n_steps_trial, dOpInf_red_model):
	"""
	solve_discrete_dOpInf_model solves the discrete dOpInf ROM for n_steps_trial over a prescribed trial time horizon

	:qhat0:            reduced initial condition
	:n_steps_trial:    number of time steps for OpInf solution
	:dOpInf_red_model: callback to the dOpInf ROM

	:return: a flag indicating NaN presence in the reduced solution and the reduced solution 
	"""

	Qtilde    	    = np.zeros((np.size(qhat0), n_steps_trial))
	contains_nans   = False

	Qtilde[:, 0] = qhat0
	for i in range(n_steps_trial - 1):
	    Qtilde[:, i + 1] = dOpInf_red_model(Qtilde[:, i])

	if np.any(np.isnan(Qtilde)):
	    contains_nans = True
            
	return contains_nans, Qtilde.T

* #### we employ a grid search to find the optimal hyperparameter values
* #### this involves exploring a range of candidate values for both $\beta_{1}$ and $\beta_{2} \in \mathbb{R}$ in a nested loop
* #### we prescribe discrete sets of candidate values $\mathcal{B}_1$ and $\mathcal{B}_2$ for the regularization parameters
* #### we consider eight candidate values for both $\mathcal{B}_1$ and $\mathcal{B}_2$, noting that in problems with more complex dynamics, sets with larger cardinalities (and bounds) might be necessary to ensure that the optimal pair leads to accurately inferred reduced operators
* #### we also prescribe a tolerance for the maximum growth of the inferred reduced coefficients over the trial time horizon, which will be used to determine the optimal regularization pair
* #### since the candidate hyperparameters are independent, this search is embarrassingly parallel
* #### the optimal hyperparameters are chosen to minimize the training error, subject to the constraint that the inferred reduced coefficient have bounded growth over a trial time horizon $[t_{\mathrm{init}}, t_{\mathrm{trial}}]$ with $t_{\mathrm{trial}} \geq t_{\mathrm{final}}$; in our implementation, the trial time horizon is the same as the target horizon, that is, $[4, 10]$ seconds
* #### in our implementation, the solution to the OpInf least-squares minimization is determined by solving the normal equations

In [ ]:
%%px
# import function to compute the Cartesian product of all candidate regularization pairs
from itertools import product

# ranges for the regularization parameter pairs
B1 = np.logspace(-10., 0., num=8)
B2 = np.logspace(-4., 4., num=8)

# threshold for the maximum growth of the inferred reduced coefficients, used for selecting the optimal regularization parameter pair 
max_growth = 1.2

# compute the Cartesian product of all regularization pairs (beta1, beta2 )
reg_pairs_global    = list(product(B1, B2))
n_reg_global        = len(reg_pairs_global)

# distribute the regularization pairs among the p MPI ranks
start_ind_reg_params, end_ind_reg_params    = \
                        distribute_reg_pairs(rank, n_reg_global, size)
reg_pairs_rank                              = \
                        reg_pairs_global[start_ind_reg_params : end_ind_reg_params]
 
# extract left and right shifted reduced data matrices for the discrete OpInf learning problem
Qhat_1 = Qhat_global.T[:-1, :]
Qhat_2 = Qhat_global.T[1:, :]

# column dimension of the reduced quadratic operator
s = int(r*(r + 1)/2)
# total column dimension of the data matrix Dhat used in the discrete OpInf learning problem
d = r + s + 1

# compute the non-redundant quadratic terms of Qhat_1 \otimes Qhat_1
Qhat_1_sq = compute_Qhat_sq(Qhat_1)

# define the constant part (due to mean shifting) in the discrete OpInf learning problem
K 	 = Qhat_1.shape[0]
Ehat = np.ones((K, 1))

# assemble the data matrix Dhat for the discrete OpInf learning problem
Dhat   = np.concatenate((Qhat_1, Qhat_1_sq, Ehat), axis=1)
# compute Dhat.T @ Dhat for the normal equations to solve the OpInf least squares minimization
Dhat_2 = Dhat.T @ Dhat

# compute the temporal mean and maximum deviation of the reduced training data
mean_Qhat_train     = np.mean(Qhat_global.T, axis=0)
max_diff_Qhat_train = np.max(np.abs(Qhat_global.T - mean_Qhat_train), axis=0)

# dictionary to store the regularization pair for each regularization pair
opt_train_err_reg_pair      = {}
# dictionary to store the reduced solutions over the target time horizon for each pair
Qtilde_dOpInf_reg_pair      = {}
# dictionary to store the OpInf ROM CPU time for each regularization pair
dOpInf_ROM_rtime_reg_pair   = {}

# loop over the regularization pairs corresponding to each MPI rank
for pair in reg_pairs_rank:

    # extract beta1 and beta2 from each candidate regularization pair
    beta1 = pair[0]
    beta2 = pair[1]

    # regularize the linear and constant reduced operators using beta1, and the reduced quadratic operator using beta2
    regg            = np.zeros(d)
    regg[:r]        = beta1
    regg[r : r + s] = beta2
    regg[r + s:]    = beta1
    regularizer     = np.diag(regg)
    Dhat_2_reg      = Dhat_2 + regularizer

    # solve the OpInf learning problem by solving the regularized normal equations
    Ohat = np.linalg.solve(Dhat_2_reg, np.dot(Dhat.T, Qhat_2)).T

    # extract the linear, quadratic, and constant reduced model operators
    Ahat = Ohat[:, :r]
    Fhat = Ohat[:, r:r + s]
    chat = Ohat[:, r + s]

    # define the discrete dOpInf reduced model 
    dOpInf_red_model    = lambda x: Ahat @ x + Fhat @ compute_Qhat_sq(x) + chat
    # extract the reduced initial condition from Qhat_1
    qhat0               = Qhat_1[0, :]
    
    # compute the reduced solution over the trial time horizon, which here is the same as the target time horizon
    start_time_dOpInf_eval          = MPI.Wtime()
    contains_nans, Qtilde_dOpInf    = solve_discrete_dOpInf_model(qhat0, nt_p, dOpInf_red_model)
    end_time_dOpInf_eval            = MPI.Wtime()

    time_dOpInf_eval = end_time_dOpInf_eval - start_time_dOpInf_eval

    # for each candidate regularization pair, we compute the training error 
    # we also save the corresponding reduced solution and ROM evaluation time
    # and compute the ratio of maximum coefficient growth in the trial period to that in the training period
    opt_train_err                               = 1e20
    opt_train_err_reg_pair[opt_train_err]       = pair
    Qtilde_dOpInf_reg_pair[opt_train_err]       = Qtilde_dOpInf
    dOpInf_ROM_rtime_reg_pair[opt_train_err]    = time_dOpInf_eval

    if contains_nans == False:
        train_err               = compute_train_err(Qhat_global.T[:nt, :], Qtilde_dOpInf[:nt, :])
        max_diff_Qhat_trial     = np.max(np.abs(Qtilde_dOpInf - mean_Qhat_train), axis=0)		
        max_growth_trial        = np.max(max_diff_Qhat_trial)/np.max(max_diff_Qhat_train)

        if max_growth_trial < max_growth:
            opt_train_err                               = train_err
            opt_train_err_reg_pair[opt_train_err]       = pair
            Qtilde_dOpInf_reg_pair[opt_train_err]       = Qtilde_dOpInf
            dOpInf_ROM_rtime_reg_pair[opt_train_err]    = time_dOpInf_eval

# find the globally minimum training error by reducing local results on each rank, subject to the bound constraint on the inferred reduced coefficients over the prescribed trial time horizon
opt_key_rank    = np.min(list(opt_train_err_reg_pair.keys()))
opt_key_rank    = np.array([opt_key_rank])
opt_key_global  = np.zeros_like(opt_key_rank)

comm.Allreduce(opt_key_rank, opt_key_global, op=MPI.MIN)
opt_key_global = opt_key_global[0]

# extract the optimal regularization pair, and the corresponding reduced solution and dOpInf ROM CPU time
if opt_key_rank == opt_key_global:
    rank_reg_opt        = rank
    Qtilde_dOpInf_opt   = Qtilde_dOpInf_reg_pair[opt_key_global]

    reg_pair_opt = opt_train_err_reg_pair[opt_key_global]

    beta1_opt = reg_pair_opt[0]
    beta2_opt = reg_pair_opt[1]

    print('\033[1m The reduced dimension that satisfies the target retained energy is: {} \033[0m'.format(r))
    print('\033[1m The optimal regularization pair was found on rank: {} \033[0m'.format(rank_reg_opt))
    print('\033[1m The optimal regularization pair is: ({}, {}) \033[0m'.format(beta1_opt, beta2_opt))

    dOpInf_ROM_rtime_opt = dOpInf_ROM_rtime_reg_pair[opt_key_global]
else:
    rank_reg_opt        = -1
    Qtilde_dOpInf_opt   = None

### Step V: Parallel postprocessing of the reduced solution
* #### in the final step, we show how to postprocess the obtained reduced solution in parallel
* #### the code below demonstrates how to use the reduced solution determined using the optimal regularization pair to compute the two approximate velocity components in the original coordinates at three probe locations positioned near the mid-channel, with increasing distance from the circular cylinder, namely $(0.40, 0.20), (0.60, 0.20)$, and $(1.00, 0.20)$
* #### the corresponding grid point indices within each snapshot for these locations are $\{16,992; 48,250; 130,722\}$
* #### let $\tilde{\mathbf{Q}} \in \mathbb{R}^{r \times 1,200}$ denote the matrix containing the reduced solution
* #### we first broadcast $\tilde{\mathbf{Q}}$ from the rank holding the optimal regularization pair to all other ranks
* #### we then map the global probe indices to local indices to identify which rank contains the respective solutions, compute the corresponding components of the local rank-$r$ POD bases, map the reduced solution for each probe to the original coordinates, and save the approximate solutions to disk

In [ ]:
%%px
target_probe_indices = [48250, 77502, 130722]

# broadcast the dOpInf reduced solution from the rank having the optimal regularization pair to all ranks
if rank == rank_reg_opt:
    for i in range(size):
        if i != rank_reg_opt:
            comm.send(Qtilde_dOpInf_opt, dest=i)
else:
    Qtilde_dOpInf_opt = comm.recv(Qtilde_dOpInf_opt, source=rank_reg_opt)

# extract and save to disk the approximate solutions at the probe locations with indices specified in target_probe_indices
for target_var_index in range(ns):
    for j, probe_index in enumerate(target_probe_indices):

        # map the global probe indices to local indices
        if probe_index >= nx_i_start and probe_index < nx_i_end:
            probe_index -= nx_i_start			

            # compute the components of the POD basis vectors corresponding to the target probe locations on each rank
            Phir_probe 			= np.matmul(Q_rank[probe_index + target_var_index*nx_i, :], Tr_global)
             # same for the temporal mean used for centering
            temporal_mean_probe = temporal_mean_rank[probe_index + target_var_index*nx_i]

            var_probe_prediction = Phir_probe @ Qtilde_dOpInf_opt.T + temporal_mean_probe

            np.save('postprocessing/dOpInf_postprocessing/dOpInf_probe_' + str(j + 1) + '_var_' + str(target_var_index + 1) + '.npy', var_probe_prediction)

print("\033[1m Rank {} reached the end of the program \033[0m".format(rank))

* #### in the final step, we plot the results
* #### the code cell below plots the POD singular values and corresponding retained energy computed with our parallel program
* #### note that the cell does not start with the parallel magic command <em>%%px</em> which means that it is executed in serial
* #### this is because we want only one rank to perform the visualization

In [ ]:
from matplotlib.pyplot import *
import matplotlib as mpl 
from matplotlib.lines import Line2D	
mpl.use('TkAgg')

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

svals = np.load('postprocessing/dOpInf_postprocessing/Sigma_sq_global.npy')

no_kept_svals_global 	= 300
no_kept_svals_energy 	= 30
no_svals_global 		= range(1, no_kept_svals_global + 1)
no_svals_energy 		= range(1, no_kept_svals_energy + 1)

retained_energy   = np.cumsum(svals)/np.sum(svals)
target_ret_energy = 0.9996

r          = np.argmax(retained_energy > target_ret_energy) + 1
ret_energy = retained_energy[r]

rcParams['lines.linewidth'] = 0
rc("figure", dpi=400)   
rc("font", family="serif")
rc("legend", edgecolor='none')
rcParams["figure.figsize"] = (6, 2)
rcParams.update({'font.size': 5})

charcoal    = [0.1, 0.1, 0.1]
color1      = '#D55E00'
color2      = '#0072B2'

fig 		= figure()
ax1 		= fig.add_subplot(121)
ax2 		= fig.add_subplot(122)

rc("figure",facecolor='w')
rc("axes",facecolor='w',edgecolor='k',labelcolor='k')
rc("savefig",facecolor='w')
rc("text",color='k')
rc("xtick",color='k')
rc("ytick",color='k')

ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.yaxis.set_ticks_position('left')
ax1.xaxis.set_ticks_position('bottom')

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.yaxis.set_ticks_position('left')
ax2.xaxis.set_ticks_position('bottom')

## plot
ax1.semilogy(no_svals_global, np.sqrt(svals)[:no_kept_svals_global]/np.sqrt(svals[0]), linestyle='-', lw=0.75, color=color1)
ax1.set_xlabel('index')
ax1.set_ylabel('singular values transformed data')

ax2.plot(no_svals_energy, retained_energy[:no_kept_svals_energy], linestyle='-', lw=0.75, color=color1)
ax2.set_xlabel('reduced dimension')
ax2.set_ylabel('% energy retained')	
ax2.plot([r, r], [0, retained_energy[r]], linestyle='--', lw=0.5, color=charcoal)
ax2.plot([0, r], [retained_energy[r], retained_energy[r]], linestyle='--', lw=0.5, color=charcoal)
##

## 
xlim = ax1.get_xlim()
ax1.set_ylim([1e-10, 1.02e0])

x_pos_all 	= np.array([0, 99, 199, 299])
labels 		= np.array([1, 100, 200, 300])
ax1.set_xticks(x_pos_all)
ax1.set_xticklabels(labels)

ax2.set_xlim([0, 30])
ax2.set_ylim([0.4, 1.001])

ax2.set_xticks([1, r, 20, 30])
ax2.set_yticks([0.5, 0.75, ret_energy , 1])
ax2.set_yticklabels([r'$50\%$', r'$75\%$', r'$99.98\%$', ''])
###

show()

* #### lastly, we plot the ROM approximate solutions at the three probe locations, $(0.40, 0.20), (0.60, 0.20)$, and $(1.00, 0.20)$
* #### the hashed areas on the left mark the training time horizon
* #### the approximate solutions are very accurate

In [ ]:
t_start = 4
t_end 	= 10
t_train = 7
dt 		= 5e-3 
t       = np.arange(t_start, t_end, dt)

ref_data_var1_loc1 = np.load('postprocessing/ref_data/ref_probe_1_var_1.npy')
ref_data_var1_loc2 = np.load('postprocessing/ref_data/ref_probe_2_var_1.npy')
ref_data_var1_loc3 = np.load('postprocessing/ref_data/ref_probe_3_var_1.npy')

ref_data_var2_loc1 = np.load('postprocessing/ref_data/ref_probe_1_var_2.npy')
ref_data_var2_loc2 = np.load('postprocessing/ref_data/ref_probe_2_var_2.npy')
ref_data_var2_loc3 = np.load('postprocessing/ref_data/ref_probe_3_var_2.npy')

dOpInf_data_var1_loc1 = np.load('postprocessing/dOpInf_postprocessing/dOpInf_probe_1_var_1.npy')
dOpInf_data_var1_loc2 = np.load('postprocessing/dOpInf_postprocessing/dOpInf_probe_2_var_1.npy')
dOpInf_data_var1_loc3 = np.load('postprocessing/dOpInf_postprocessing/dOpInf_probe_3_var_1.npy')

dOpInf_data_var2_loc1 = np.load('postprocessing/dOpInf_postprocessing/dOpInf_probe_1_var_2.npy')
dOpInf_data_var2_loc2 = np.load('postprocessing/dOpInf_postprocessing/dOpInf_probe_2_var_2.npy')
dOpInf_data_var2_loc3 = np.load('postprocessing/dOpInf_postprocessing/dOpInf_probe_3_var_2.npy')

rcParams['lines.linewidth'] = 0
rc("figure", dpi=400)           
rc("font", family="serif")      
rc("legend", edgecolor='none')
rcParams["figure.figsize"] = (8, 4)
rcParams.update({'font.size': 6})

charcoal    = [0.1, 0.1, 0.1]
color1      = '#D55E00'
color2      = '#0072B2'

fig 		= figure()
ax11 		= fig.add_subplot(231)
ax12 		= fig.add_subplot(232, sharex=ax11, sharey=ax11)
ax13 		= fig.add_subplot(233, sharex=ax11, sharey=ax11)
ax21 		= fig.add_subplot(234)
ax22 		= fig.add_subplot(235, sharex=ax21, sharey=ax21)
ax23 		= fig.add_subplot(236, sharex=ax21, sharey=ax21)

rc("figure",facecolor='w')
rc("axes",facecolor='w',edgecolor='k',labelcolor='k')
rc("savefig",facecolor='w')
rc("text",color='k')
rc("xtick",color='k')
rc("ytick",color='k')

ax11.spines['right'].set_visible(False)
ax11.spines['top'].set_visible(False)
ax11.yaxis.set_ticks_position('left')
ax11.xaxis.set_ticks_position('bottom')

ax12.spines['right'].set_visible(False)
ax12.spines['top'].set_visible(False)
ax12.yaxis.set_ticks_position('left')
ax12.xaxis.set_ticks_position('bottom')

ax21.spines['right'].set_visible(False)
ax21.spines['top'].set_visible(False)
ax21.yaxis.set_ticks_position('left')
ax21.xaxis.set_ticks_position('bottom')

ax22.spines['right'].set_visible(False)
ax22.spines['top'].set_visible(False)
ax22.yaxis.set_ticks_position('left')
ax22.xaxis.set_ticks_position('bottom')

ax13.spines['right'].set_visible(False)
ax13.spines['top'].set_visible(False)
ax13.yaxis.set_ticks_position('left')
ax13.xaxis.set_ticks_position('bottom')

ax23.spines['right'].set_visible(False)
ax23.spines['top'].set_visible(False)
ax23.yaxis.set_ticks_position('left')
ax23.xaxis.set_ticks_position('bottom')

## plot
ax11.plot(t, ref_data_var1_loc1, linestyle='-', lw=1.00, color=charcoal)
ax11.plot(t, dOpInf_data_var1_loc1, linestyle='--', lw=1.00, color=color1)

ax12.plot(t, ref_data_var1_loc2, linestyle='-', lw=1.00, color=charcoal)
ax12.plot(t, dOpInf_data_var1_loc2, linestyle='--', lw=1.00, color=color1)

ax13.plot(t, ref_data_var1_loc3, linestyle='-', lw=1.00, color=charcoal)
ax13.plot(t, dOpInf_data_var1_loc3, linestyle='--', lw=1.00, color=color1)


ax21.plot(t, ref_data_var2_loc1, linestyle='-', lw=1.00, color=charcoal)
ax21.plot(t, dOpInf_data_var2_loc1, linestyle='--', lw=1.00, color=color1)

ax22.plot(t, ref_data_var2_loc2, linestyle='-', lw=1.00, color=charcoal)
ax22.plot(t, dOpInf_data_var2_loc2, linestyle='--', lw=1.00, color=color1)

ax23.plot(t, ref_data_var2_loc3, linestyle='-', lw=1.00, color=charcoal)
ax23.plot(t, dOpInf_data_var2_loc3, linestyle='--', lw=1.00, color=color1)
##

ax11.axvline(x=t_train, lw=1.00, linestyle='--', color='gray')
ax12.axvline(x=t_train, lw=1.00, linestyle='--', color='gray')
ax13.axvline(x=t_train, lw=1.00, linestyle='--', color='gray')
ax21.axvline(x=t_train, lw=1.00, linestyle='--', color='gray')
ax22.axvline(x=t_train, lw=1.00, linestyle='--', color='gray')
ax23.axvline(x=t_train, lw=1.00, linestyle='--', color='gray')

ax11.set_title('probe 1 (0.4, 0.2)')
ax12.set_title('probe 2 (0.6, 0.2)')
ax13.set_title('probe 3 (1.0, 0.2)')

ax11.set_ylabel(r'$u_x$')
ax21.set_ylabel(r'$u_y$')

fig.supxlabel('target time horizon (seconds)')

xlim = ax11.get_xlim()
ax11.set_xlim([xlim[0], 10])
ax21.set_xlim([xlim[0], 10])

ylim = ax11.get_ylim()
ax11.set_ylim([ylim[0], 1.25])

ylim = ax21.get_ylim()
ax21.set_ylim([ylim[0], 0.4])

rect = Rectangle((0, 0), width=t_train, height=1.25, hatch='/', color='grey', alpha=0.2, label='training region')
ax11.add_patch(rect)
rect = Rectangle((0, 0), width=t_train, height=1.25, hatch='/', color='grey', alpha=0.2, label='training region')
ax12.add_patch(rect)
rect = Rectangle((0, 0), width=t_train, height=1.25, hatch='/', color='grey', alpha=0.2, label='training region')
ax13.add_patch(rect)

rect = Rectangle((0, -0.4), width=t_train, height=0.8, hatch='/', color='grey', alpha=0.2, label='training region')
ax21.add_patch(rect)
rect = Rectangle((0, -0.4), width=t_train, height=0.8, hatch='/', color='grey', alpha=0.2, label='training region')
ax22.add_patch(rect)
rect = Rectangle((0, -0.4), width=t_train, height=0.8, hatch='/', color='grey', alpha=0.2, label='training region')
ax23.add_patch(rect)

x_pos_all 	= np.array([4, 5, 6, 7, 8, 9, 10])
labels 		= np.array([4, 5, 6, 7, 8, 9, 10])
ax11.set_xticks(x_pos_all)
ax11.set_xticklabels(labels)
ax21.set_xticks(x_pos_all)
ax21.set_xticklabels(labels)

show()